In [122]:
%matplotlib inline
import torch
import torchvision
from IPython import display
from matplotlib import pyplot as plt
import numpy as np
import random

import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
import tqdm

In [123]:
class AvazuDataSet(torch.utils.data.Dataset):
    def __init__(self, file_path):
        self.data = pd.read_csv(file_path, index_col=False, sep=',')
        self.len = len(self.data)
        lable = 'click'
        sparse_features = ['id', 'hour', 'C1', 'banner_pos', 
                   'site_id', 'site_domain','site_category', 
                   'app_id', 'app_domain', 'app_category', 
                   'device_id','device_ip', 'device_model', 'device_type', 'device_conn_type', 
                   'C14','C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21']
        #print(self.data.columns)
        self.feat_dims = []
        for feat in sparse_features:
            lbe = LabelEncoder()
            self.data[feat] = lbe.fit_transform(self.data[feat])
            self.feat_dims.append(len(lbe.classes_))
            
        self.features = self.data[sparse_features]
        self.targets = self.data[lable]

    def __len__(self):
        return self.len

    def __getitem__(self, index):
        return torch.tensor(self.features.iloc[index].to_numpy(), dtype=torch.long), torch.tensor(self.targets.iloc[index], dtype=torch.float32)
    
    def get_feat_dims(self):
        return self.feat_dims

In [124]:
data_set = CriteoDataSet('../data/avazu_sample.txt')

data_loader = torch.utils.data.DataLoader(data_set, batch_size=8, shuffle=True)
for x, y in data_loader:
    print(x, y)
    print(x.shape, y.shape)
    break
    
feat_dims = data_set.get_feat_dims()
print(feat_dims)

tensor([[159,   0,   2,   0,  70,  86,   4,  64,   5,   1,  86, 893, 171,   1,
           0,   4,   2,   1,   3,   2,   1,  35,   4],
        [370,   0,   2,   0,  21,  71,   8,  69,   6,   0,  86, 151, 177,   1,
           0,   4,   2,   1,   3,   2,   1,  35,   4],
        [789,   0,   2,   0,  76,  50,   3,  69,   6,   0,  86, 565, 171,   1,
           0,  42,   1,   2,  32,   2,   1,  26,   5],
        [607,   0,   2,   1, 105,  55,   8,  69,   6,   0,  86,  80, 247,   1,
           0,   4,   2,   1,   3,   2,   1,  35,   4],
        [542,   0,   2,   0, 114,  38,   8,  69,   6,   0,  86, 614,  27,   1,
           0,  81,   2,   1,  40,   0,   0,   0,  20],
        [262,   0,   2,   0,  17, 103,   1,  69,   6,   0,  86, 321, 155,   1,
           0,  22,   2,   1,  14,   0,   0,   0,  12],
        [470,   0,   2,   0,  70,  86,   4,  43,  14,   1,  86, 707, 324,   1,
           0,  88,   2,   1,  55,   0,  16,  49,   7],
        [646,   0,   2,   0,  70,  86,   4,  22,   1,   1,  86

In [125]:
class FeaturesLinear(torch.nn.Module):

    def __init__(self, field_dims, output_dim=1):
        super().__init__()
        self.fc = torch.nn.Embedding(sum(field_dims), output_dim)
        self.bias = torch.nn.Parameter(torch.zeros((output_dim,)))
        self.offsets = np.array((0, *np.cumsum(field_dims)[:-1]), dtype=np.long)

    def forward(self, x):
        """
        :param x: Long tensor of size ``(batch_size, num_fields)``
        """
        x = x + x.new_tensor(self.offsets).unsqueeze(0)
        return torch.sum(self.fc(x), dim=1) + self.bias


class FeaturesEmbedding(torch.nn.Module):

    def __init__(self, field_dims, embed_dim):
        super().__init__()
        self.embedding = torch.nn.Embedding(sum(field_dims), embed_dim)
        self.offsets = np.array((0, *np.cumsum(field_dims)[:-1]), dtype=np.long)
        torch.nn.init.xavier_uniform_(self.embedding.weight.data)

    def forward(self, x):
        """
        :param x: Long tensor of size ``(batch_size, num_fields)``
        """
        x = x + x.new_tensor(self.offsets).unsqueeze(0)
        return self.embedding(x)
    
class FactorizationMachine(torch.nn.Module):

    def __init__(self, reduce_sum=True):
        super().__init__()
        self.reduce_sum = reduce_sum

    def forward(self, x):
        """
        :param x: Float tensor of size ``(batch_size, num_fields, embed_dim)``
        """
        square_of_sum = torch.sum(x, dim=1) ** 2
        sum_of_square = torch.sum(x ** 2, dim=1)
        ix = square_of_sum - sum_of_square
        if self.reduce_sum:
            ix = torch.sum(ix, dim=1, keepdim=True)
        return 0.5 * ix

In [126]:
class FactorizationMachineModel(torch.nn.Module):
    """
    A pytorch implementation of Factorization Machine.

    Reference:
        S Rendle, Factorization Machines, 2010.
    """

    def __init__(self, field_dims, embed_dim):
        super().__init__()
        self.embedding = FeaturesEmbedding(field_dims, embed_dim)
        self.linear = FeaturesLinear(field_dims)
        self.fm = FactorizationMachine(reduce_sum=True)

    def forward(self, x):
        """
        :param x: Long tensor of size ``(batch_size, num_fields)``
        """
        x = self.linear(x) + self.fm(self.embedding(x))
        return torch.sigmoid(x.squeeze(1))

In [127]:
model = FactorizationMachineModel(feat_dims, 16)
print(model)

FactorizationMachineModel(
  (embedding): FeaturesEmbedding(
    (embedding): Embedding(3013, 16)
  )
  (linear): FeaturesLinear(
    (fc): Embedding(3013, 1)
  )
  (fm): FactorizationMachine()
)


In [128]:
for name, p in model.named_parameters():
    print(name)
    print(p.shape)

embedding.embedding.weight
torch.Size([3013, 16])
linear.bias
torch.Size([1])
linear.fc.weight
torch.Size([3013, 1])


In [129]:
### Train
def run_train(model, optimizer, data_loader, criterion, device, log_interval=1000):
    model.train()
    total_loss = 0
    for i, (fields, target) in enumerate(tqdm.tqdm(data_loader, smoothing=0, mininterval=1.0)):
        fields, target = fields.to(device), target.to(device)
        y = model(fields)
        loss = criterion(y, target.float())
        model.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        if (i + 1) % log_interval == 0:
            print('    - loss:', total_loss / log_interval)
            total_loss = 0

def run_test(model, data_loader, device):
    model.eval()
    targets, predicts = list(), list()
    with torch.no_grad():
        for fields, target in tqdm.tqdm(data_loader, smoothing=0, mininterval=1.0):
            fields, target = fields.to(device), target.to(device)
            y = model(fields)
            targets.extend(target.tolist())
            predicts.extend(y.tolist())
    return roc_auc_score(targets, predicts)

In [130]:
train_length = int(len(data_set) * 0.8)
valid_length = int(len(data_set) * 0.1)
test_length = len(data_set) - train_length - valid_length
train_dataset, valid_dataset, test_dataset = torch.utils.data.random_split(
    data_set, (train_length, valid_length, test_length))
print(len(train_dataset), len(valid_dataset), len(test_dataset))

799 99 101


In [131]:
# config
config = {
    'batch_size': 32,
    'lr':1e-3,
    'weight_decay': 1e-6,
    'epoch':3,
    'device':'cpu'
}
device = torch.device(config['device')

In [132]:
train_data_loader = torch.utils.data.DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True,num_workers=8)
valid_data_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=config['batch_size'], shuffle=True, num_workers=8)
test_data_loader = torch.utils.data.DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=True, num_workers=8)

criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=config['lr'], weight_decay=config['weight_decay'])

In [133]:
model = FactorizationMachineModel(feat_dims, 16)

In [134]:
for epoch_i in range(config['epoch']):
    run_train(model, optimizer, train_data_loader, criterion, device)
    auc = run_test(model, valid_data_loader, device)
    print('epoch:', epoch_i, 'validation: auc:', auc)

  0%|          | 0/25 [00:00<?, ?it/s]

epoch: 0 validation: auc: 0.5566714490674318


  0%|          | 0/25 [00:00<?, ?it/s]

epoch: 1 validation: auc: 0.5566714490674318


100%|██████████| 4/4 [00:00<00:00, 30.80it/s]

epoch: 2 validation: auc: 0.5566714490674318
